In [ ]:
# Coded by G. Petricca (@gmrpetricca)

from obspy.clients.fdsn import Client
from obspy import UTCDateTime

from matplotlib.colors import rgb2hex
import matplotlib.colors as colors
import matplotlib.image as image
import matplotlib.pyplot as plt

from pylab import cm
import numpy as np

# start user-efined section
# setup station information
LOCATION = "" # station location
NETWORK = ""   # AM = RaspberryShake network
STATION = ""  # station code to get data from
CHANNEL = "F"  # channel to grab data for (e.g. EHZ, SHZ, EHE, EHN)
LOC = "" # loc of the station (usually 00)
STA_LAT = XX.XX  # latitude of station  
STA_LON = XX.XX  # longitude of station
TIME_ZONE = "" # time zone of station

# define the date/time for the plot starting point and the lenght of the plot
record_start="2020-12-28 00:00:00"
time_lenght=24 # in hours

# define details for the filter
F1 = 0.7 # high-pass filter corner
F2 = 2.0 # low-pass filter corner 
cor = 4 # number of corners
# end user-defined section

# set-up figure
fig = plt.figure(figsize=(30, 20))

# convert this into UTC Date Time format
starttime = UTCDateTime(record_start)

# calculate endtime from UTC starttime
endtime = starttime + (time_lenght*60*60)

# define fdsn client to get data from
client = Client('https://fdsnws.raspberryshakedata.com')

# download and filter the data
st = client.get_waveforms(NETWORK, STATION, LOC, CHANNEL, starttime, endtime)
st.detrend(type='demean')
st.filter("bandpass", freqmin=F1, freqmax=F2, corners=cor)

# select appropriate colors from an existing colormap
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

cmap = plt.get_cmap('viridis')
def_cmap = truncate_colormap(cmap, 0.2, 0.8)

colormap = cm.get_cmap(def_cmap, 5)

plotcolors = []
for i in range(colormap.N):
    rgba = colormap(i)
    hex_code = rgb2hex(rgba)
    plotcolors.append(hex_code)

# take only four colors from the colormap, equally spaced 
plotcolors = plotcolors[0::80]

# select appropriate scaling range based on channel
if CHANNEL == "HDF":
    vert = 50000
else:
    vert = 400

# create the daily plot
st.plot(type="dayplot", interval=60, right_vertical_labels=False, one_tick_per_line=True, color=plotcolors, show_y_UTC_label=False, vertical_scaling_range=vert, fig=fig, title=None, x_labels_size=12, y_labels_size=12, number_of_ticks=13, linewidth=1.2, localization_dict={'time in': '', 'seconds': '', 'minutes': '', 'hours': ''})

# infographic title and text
if CHANNEL == "HDF":
    fig.text(0.50, 0.980, "Infrasound Signal recorded from: " + str(LOCATION) + " - Lat: "+str(STA_LAT)+"° - Lon: "+str(STA_LON)+"° - Raspberry Shake Network ID: " + STATION + " - Channel: "+str(CHANNEL)+" - Bandpass Filter: "+str(F1)+" - "+str(F2)+" Hz", ha="center", va="center", size=19 , color="darkblue")
else:    
    fig.text(0.50, 0.980, "Seismic Signal recorded from: " + str(LOCATION) + " - Lat: "+str(STA_LAT)+"° - Lon: "+str(STA_LON)+"° - Raspberry Shake Network ID: " + STATION + " - Channel: "+str(CHANNEL)+" - Bandpass Filter: "+str(F1)+" - "+str(F2)+" Hz", ha="center", va="center", size=19 , color="darkblue")

fig.text(0.0315, 0.035, "Time in Minutes", color="darkblue", fontsize=14)
fig.text(0.033, 0.950, "Start day/time: " + str(record_start), ha="left", va="center", color="black", fontsize=13)
fig.text(0.9925, 0.950, "Duration: " + str(time_lenght) + " hours", ha="right", va="center", color="black", fontsize=13)
fig.text(0.011, 0.936, TIME_ZONE, color="black", fontsize=12)

# infographic signature
fig.text(0.993, 0.025, "Infographic by:", ha="right", va="center", size=12 ,color="black")
fig.text(0.993, 0.01, "Giuseppe Petricca         @gmrpetricca", ha="right", va="center", size=12 ,color="black")

# figure additional logos
im1 = image.imread('logos/HWlogo.png') # if you have a personal logo, you can add it here
fig.figimage(im1, 955, 0, zorder=3)

im2 = image.imread('logos/matplotliblogo.png')
fig.figimage(im2, 1125, 12, zorder=3)

im3 = image.imread('logos/obspylogo.png')
fig.figimage(im3, 775, 15, zorder=3)

im4 = image.imread('logos/RSlogo.png')
fig.figimage(im4, 1305, 15, zorder=3)

im4 = image.imread('logos/twitterlogo.png')
fig.figimage(im4, 2035, 4, zorder=3)

# save and plot the figure
plt.tight_layout(rect=[0, 0.03, 1.0, 0.95])
plt.savefig(STATION + "_" + record_start.replace(" ", "_").replace(":","-") + '_' + str(time_lenght) + 'h_' + CHANNEL + 'DailyPlot.png')
plt.show()